# Single Responsibility Principle

The **Single Responsibility Principle (SRP)** is one of the five SOLID principles of Object-Oriented Design. It states:
> "A class should have only one reason to change".

This means that a class should have **only one job** or **one responsibility** in the system. If a class is handling multiple responsibilities, it becomes harder to maintain and modify.

## Why is SRP Important?

* ✅ Makes code easier to understand
* ✅ Simplifies debugging and testing
* ✅ Reduces unnecessary dependencies
* ✅ Improves scalability and flexibility

## Simple Example (Bad vs Good Code)

Consider the scenario where you are building a Report Generation System for a company. The system needs to:

1. Generate a report containing some structured data.
2. Save the report to a file for future reference.
3. Print the report in a readable format for users.


### ❌ Bad Code (Violates SRP)

```python
class Report:
    def __init__(self, data):
        self.data = data

    def generate_report(self):
        """Generates a simple text report."""
        return f"Report Data: {self.data}"

    def save_to_file(self, filename):
        """Saves the report to a file."""
        with open(filename, "w") as file:
            file.write(self.generate_report())

    def print_report(self):
        """Prints the report in a simple format."""
        print(self.generate_report())
```



#### 🛑 The Problem: Code is Tightly Coupled



* Suppose we change how reports are formatted, for example, switching from plain text to JSON:

    ```python
    def generate_report(self):
        """Now generates a JSON-formatted report."""
        return {"report_data": self.data}  # Returns a dictionary instead of a string
    ```



##### 🔴 Issue



* Since `generate_report()` now returns a dictionary instead of a string, both `print_report()` and `save_to_file()` will break because they expect text.
* Saving to a file will fail (`file.write()` requires a string, not a dictionary).
* Printing will show an incorrect format instead of readable text.



##### 🔴 What’s wrong?



* The Report class has multiple responsibilities:
    * Generating the report (generate_report())
    * Saving the report (save_to_file())
    * Printing the report (print_report())

* If we need to change how we save files, we also affect report generation and printing—this violates SRP!

### ✅ Good Code (Follows SRP)

```python
import json

class Report:
    def __init__(self, data):
        self.data = data

    def generate_report(self):
        """Returns structured data (not responsible for formatting)."""
        return {"report_data": self.data}

class ReportSaver:
    def __init__(self, report):
        self.report = report

    def save_to_file(self, filename):
        """Saves the report in a file as JSON."""
        with open(filename, "w") as file:
            file.write(json.dumps(self.report.generate_report()))

class ReportPrinter:
    def __init__(self, report):
        self.report = report

    def print_report(self):
        """Prints the report in a formatted way."""
        print(json.dumps(self.report.generate_report(), indent=4))
```



#### 🟢 Why is this Better?


* ✅ Each class has a single responsibility:
    * `Report` → Only generates data (not formatting, saving, or printing).
    * `ReportSaver` → Handles file saving (without modifying how reports are generated).
    * `ReportPrinter` → Handles printing (without affecting saving).

* ✅ Now, if we change generate_report(), it won’t break the saving or printing logic.

* ✅ Code is more reusable and easier to maintain!

### Detailed Analysis (Optional)

#### 1️⃣ Each Class Has a Single Responsibility

|      Class      |                               Responsibility                              |                                                     Why It’s Good?                                                    |
|:---------------:|:-------------------------------------------------------------------------:|:---------------------------------------------------------------------------------------------------------------------:|
| `Report`        | Only generates data (not concerned with formatting, saving, or printing). | If we change the report format (e.g., switch to XML instead of JSON), it won’t affect saving or printing.             |
| `ReportSaver`   | Saves the report to a file.                                               | If we change the file format (e.g., save as `.csv` instead of `.json`), only `ReportSaver` needs to be modified.      |
| `ReportPrinter` | Handles printing the report.                                              | If we decide to print reports in a different format (e.g., HTML instead of plain text), only `ReportPrinter` changes. |

**🔴 What happens in the bad code?**

* One big class (`Report`) is responsible for multiple tasks.
* A change in one method affects others.
* Difficult to modify and test without breaking something else.


#### 2️⃣ Code is Easier to Modify and Extend

Let’s say in the future, we need to:

1. Save the report to **a database instead of a file.**
2. Print reports as **HTML instead of JSON.**

**Bad Code**

* We would have to **modify the `Report` class directly**, affecting multiple functions.
* We risk breaking both **saving and printing functionality** while making small changes.
* The class becomes **harder to read and maintain.**

**Good Code (Following SRP)**

* We **only modify `ReportSaver` to support databases** without affecting the `Report` class.
* We **only modify `ReportPrinter` to support HTML** without affecting saving.
* The `Report` class remains **unchanged**, ensuring stability.

**💡 Key Benefit**

The improved code is easier to modify because each class only handles one thing.


#### 3️⃣ Easier to Test and Debug

* In the **bad version**, if a test fails, we have to check the entire `Report` class for errors, since everything (data generation, saving, printing) is mixed together.

* In the **good version**, we can **test each class independently**:
    * `Report` → Check if it generates the correct data.
    * `ReportSaver` → Check if the file is saved correctly.
    * `ReportPrinter` → Check if it prints correctly.

**✅ Less debugging, faster development, fewer bugs! 🎯**


#### 4️⃣ Improved Reusability

* In the bad version, if another part of our application needs to **save reports differently** (e.g., to a cloud service), we would have to **modify the existing class**.
* In the **good version**, we can simply **create a new `CloudReportSaver` class** without touching existing logic.

**✅ We can reuse `ReportPrinter` and `ReportSaver` in different parts of the project!**


#### 🚀 Final Takeaway
✅ The **bad version is tightly coupled**, meaning small changes cause **unexpected breakages**.
✅ The **good version is modular**, meaning small changes are **localized and predictable**.
✅ This makes the **good version scalable, maintainable, and reusable**.

By following the **Single Responsibility Principle (SRP)**, we ensure that each part of the system has one clear purpose, making it **simpler, more reliable, and easier to expand in the future. 🎯🚀**

---

## Practice Problem: Refactor an Image Processing System

### Background
You are working on an **image processing system** for a startup that applies filters to images, saves them, and logs operations. However, the current implementation is becoming **difficult to maintain** because a single class is handling multiple responsibilities.

Your task is to **refactor the existing code** to follow the **Single Responsibility Principle (SRP)** while ensuring the system remains **extensible and maintainable.**

### 📝 Current Implementation (Needs Refactoring)

The `ImageProcessor` class is responsible for:

* Loading images
* Applying filters
* Saving images
* Logging operations

Having multiple responsibilities in a single class makes it **harder to modify, test, and extend.**

#### ❌ Current Code (Violating SRP)

```python
class ImageProcessor:
    def __init__(self, image_path):
        self.image_path = image_path
        self.image = None

    def load_image(self):
        """Loads an image (simulation)."""
        self.image = f"Image loaded from {self.image_path}"
        return self.image

    def apply_filter(self, filter_type):
        """Applies a filter to the image."""
        if self.image is None:
            raise ValueError("No image loaded!")
        self.image += f" | Applied {filter_type} filter"
        return self.image

    def save_image(self, output_path):
        """Saves the image (simulation)."""
        if self.image is None:
            raise ValueError("No image to save!")
        with open(output_path, "w") as file:
            file.write(self.image)

    def log_operation(self, message):
        """Logs an operation."""
        print(f"[LOG]: {message}")


# ---- Example Usage ----
processor = ImageProcessor("input.jpg")
processor.load_image()
processor.apply_filter("grayscale")
processor.save_image("output.jpg")
processor.log_operation("Image processing completed.")
```

### 🎯 Your Task: Refactor the Code

You need to **separate responsibilities** so that each class has only one reason to change.

* 🔹 **Break down the `ImageProcessor` class into distinct components:**

    * `ImageLoader` → Responsible for loading images.
    * `ImageFilter` → Responsible for applying filters.
    * `ImageSaver` → Handles saving the image.
    * `Logger` → Manages logging operations.

* 🔹 Ensure that:

    * Each class is **independent** and handles **only one task**.
    * The system remains **scalable and easy to extend.**
    * Adding new filters or changing the storage system should not affect other components.
    * In order for the test cases to pass, retain the same code logic inside each newly created component.

### ✅ Expected Usage (After Refactoring)

Once refactored, your code should work like this:

```python
image = ImageLoader("input.jpg").load()
filtered_image = ImageFilter(image).apply("grayscale")
ImageSaver(filtered_image).save("output.jpg")
Logger().log("Image processing completed.")
```

* The `ImageLoader` class should only **load** the image.
* The `ImageFilter` class should only **apply filters**.
* The `ImageSaver` class should only **handle saving**.
* The `Logger` class should only **log messages**.

In [ ]:
# 📌 Implement the 4 components here
# This is just a placeholder so that test cases can run


# ImageLoader Responsible for loading images.
class ImageLoader:

# ImageFilter Responsible for applying filters.
class ImageFilter:

# ImageSaver Handles saving the image.
class ImageSaver:

# Logger Manages logging operations.
class Logger:


#### ✅ Test Cases

In [ ]:
# 🚀 Run this cell to validate the refactored implementation

def test_image_loader():
    print("Testing ImageLoader...")
    loader = ImageLoader("test.jpg")
    image = loader.load()
    assert image == "Image loaded from test.jpg", "❌ ImageLoader failed"
    print("✅ ImageLoader passed!")

def test_image_filter():
    print("Testing ImageFilter...")
    image = "Image loaded from test.jpg"
    filtered_image = ImageFilter(image).apply("grayscale")
    assert "Applied grayscale filter" in filtered_image, "❌ ImageFilter failed"
    print("✅ ImageFilter passed!")

def test_image_saver():
    print("Testing ImageSaver...")
    image = "Processed Image Data"
    saver = ImageSaver(image)
    output_path = "output_test.txt"
    
    saver.save(output_path)
    
    assert os.path.exists(output_path), "❌ ImageSaver failed"
    
    os.remove(output_path)  # Cleanup test file
    print("✅ ImageSaver passed!")

def test_logger():
    print("Testing Logger...")
    logger = Logger()
    logger.log("Test message")  # Expected output: [LOG]: Test message
    print("✅ Logger passed!")

def test_full_pipeline():
    print("Testing Full Image Processing Pipeline...")
    
    image = ImageLoader("test.jpg").load()
    filtered_image = ImageFilter(image).apply("sepia")
    
    output_path = "final_output.txt"
    ImageSaver(filtered_image).save(output_path)
    
    assert os.path.exists(output_path), "❌ Full pipeline failed"
    
    os.remove(output_path)
    print("✅ Full pipeline passed!")

# Run all tests
test_image_loader()
test_image_filter()
test_image_saver()
test_logger()
test_full_pipeline()

## 🎯 Final Conclusion

By completing this exercise, you've successfully refactored a **poorly structured** image processing system into a **modular and maintainable** design following the **Single Responsibility Principle (SRP)**.

### 🔑 Key Takeaways

* ✅ **Separation of Concerns** – Each class now has a single responsibility, making the **code easier to read, modify, and extend**.
* ✅ **Scalability & Maintainability** – New features (e.g., additional filters or cloud storage) can be added **without breaking existing functionality**.
* ✅ **Test-Driven Approach** – The provided test cases ensure that your solution is **functionally correct** and follows **best coding practices**.
* ✅ **Real-World Application** – This principle applies to large-scale software projects where **clean architecture** is essential for long-term success.

### 🚀 Interview Perspective

Understanding **SRP** is **crucial for Low-Level Design (LLD) interviews**. Many interviewers assess your **ability to break down complex systems into well-structured, maintainable components**.

#### 📌 How SRP Helps in Interviews
* 🔹 **System Design Questions** – You’ll often be asked to **design a system from scratch** (e.g., a file storage system, a payment gateway, or a ride-sharing app). Demonstrating SRP will show that you **think modularly**.

* 🔹 **Refactoring Challenges** – Some interviews give **badly structured code** and ask you to refactor it. The ability to **identify violations of SRP** and improve the design is highly valued.

* 🔹 **Code Maintainability Discussions** – When explaining your solutions, always highlight **how SRP helps avoid unintended side effects** when requirements change.

### 🚀 Next Steps

* 🔹 Try applying **SRP** in your own projects and **identify violations** in existing codebases.
* 🔹 Study **other SOLID principles** (OCP, LSP, ISP, DIP) to enhance your **object-oriented design thinking**.
* 🔹 Solve **real-world LLD problems** (e.g., design a URL shortener, a notification system, a task scheduler).

Mastering **SRP and other design principles** will **set you apart in interviews** and help you build **robust, scalable software systems** in your career! 🚀🔥